<a href="https://colab.research.google.com/github/sugiyama404/ReinfoceLearningForTrading/blob/main/a2c_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime
import random
import copy
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

import math

from google.colab import drive

mode = 'test'
name = 'a2c'

drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
nov_path = '/content/drive/My Drive/' + nov_dir + f'sp500_{mode}.csv'

exp_dir = 'Colab Notebooks/workspace/export/'
mdl_dir = '/content/drive/My Drive/' + exp_dir + 'models'
csv_path = '/content/drive/My Drive/' + exp_dir + f'csv_data/{name}_{mode}.csv'

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

Mounted at /content/drive/


In [2]:
class Environment:
    def __init__(self, df, initial_money=100000, mode = 'test'):

        self.df = df.dropna().reset_index()
        self.df_total_steps = len(self.df)-1
        self.initial_money = initial_money
        self.mode = mode
        self.trade_time = None
        self.trade_win = None
        self.brfore_buy_cash = None
        self.action_space = np.array([0, 1, 2]) # buy,hold,sell
        self.hold_a_position = None
        self.now_price = None
        self.cash_in_hand = None

        self.reset()
        
    def reset(self):

        self.trade_time      = 0
        self.trade_win       = 0
        self.brfore_buy_cash = 0
        self.end_step        = self.df_total_steps
        self.now_step        = 0
        self.hold_a_position = 0.0
        self.now_price       = self.df.loc[self.now_step, 'SP500']
        self.cash_in_hand    = self.initial_money

        return self._get_now_state()

    def step(self, action):

        prev_revenue = self._get_revenue()
        self.now_step += 1
        self.now_price = self.df.loc[self.now_step, 'SP500']
 
        done = (self.end_step == self.now_step)

        self._trade(action,done)
        cur_revenue = self._get_revenue()
 
        reward = cur_revenue - prev_revenue

        if self.mode == 'test':
            info = { 'cur_revenue' : cur_revenue , 'trade_time' : self.trade_time, 'trade_win' : self.trade_win }
        else:
            info = { 'cur_revenue' : cur_revenue }

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        state = np.empty(3)
        state[0] = self.hold_a_position
        state[1] = self.now_price
        state[2] = self.cash_in_hand
        return state

    def _get_revenue(self): 
        return self.hold_a_position * self.now_price + self.cash_in_hand

    def _trade(self, action,lastorder = False):
        if lastorder:
            self.cash_in_hand += self.now_price * self.hold_a_position
            self.hold_a_position = 0
            if self.mode == 'test':
                self.trade_time += 1
                if self.cash_in_hand > self.brfore_buy_cash:
                    self.trade_win += 1
        else:
            if self.action_space[0] == action: # buy
                if self.hold_a_position == 0:
                    buy_flag = True
                    if self.mode == 'test':
                        self.brfore_buy_cash = copy.copy(self.cash_in_hand)
                    while buy_flag:
                        if self.cash_in_hand > self.now_price:
                            self.hold_a_position += 1
                            self.cash_in_hand -= self.now_price
                        else:
                            buy_flag = False
            if self.action_space[2] == action: # sell
                if self.hold_a_position != 0:
                    self.cash_in_hand += self.now_price * self.hold_a_position
                    self.hold_a_position = 0
                    if self.mode == 'test':
                        self.trade_time += 1
                        if self.cash_in_hand > self.brfore_buy_cash:
                            self.trade_win += 1

In [3]:
class Brain:
    def __init__(self):

        n_shape = 3
        lr = 0.01

        common = input_ = keras.layers.Input(shape=n_shape)
        common = keras.layers.Dense(128, activation="relu")(common)

        actor = keras.layers.Dense(3, activation="softmax")(common)
        critic = keras.layers.Dense(1, activation="linear")(common)

        model = keras.Model(input_, [actor, critic])
        model.compile(optimizer=Adam(lr=lr))
        model.summary()
        Brain.model = model


    def load(self, name):
        Brain.model.load_weights(name)

    def save(self, name):
        Brain.model.save_weights(name)

In [4]:
class Actor(Brain):
    def __init__(self):
        super().__init__()

    def policynetwork(self, state):
        act_p, _ = Brain.model(state.reshape((1,-1)))
        return np.random.choice(3, p=act_p[0].numpy())

In [5]:
class Critic(Brain):
    def __init__(self):

        self.gamma = 0.9
        self.beta  = 0.1

    def valuenetwork(self, experiences):

        discounted_return = self._discounted_return(experiences)

        state_batch = np.asarray([e["state"] for e in experiences])
        action_batch = np.asarray([e["action"] for e in experiences])

        onehot_actions = tf.one_hot(action_batch, 3)

        with tf.GradientTape() as tape:

            act_p, v = Brain.model(state_batch, training=True)
            selct_pai = tf.reduce_sum(onehot_actions * act_p, axis=1, keepdims=True)
            selected_action_probs = tf.clip_by_value(selct_pai, 1e-10, 1.0)
            advantage = discounted_return - tf.stop_gradient(v)

            value_losses = self._value_losses(advantage)
            policy_losses = self._policy_losses(advantage,selected_action_probs,v,discounted_return)
            total_loss = value_losses + policy_losses
            loss = tf.reduce_mean(total_loss)

        gradients = tape.gradient(loss, Brain.model.trainable_variables)

        Brain.model.optimizer.apply_gradients(
            (grad, var) 
            for (grad, var) in zip(gradients, Brain.model.trainable_variables) 
            if grad is not None
        )

    def _discounted_return(self,experiences):
        if experiences[-1]["done"]:
            G = 0
        else:
            next_state = np.atleast_2d(experiences[-1]["next_state"])
            _, n_v = Brain.model(next_state)
            G = n_v[0][0].numpy()

        discounted_return = []
        for exp in reversed(experiences):
            if exp["done"]:
                G = 0
            G = exp["reward"] + self.gamma * G
            discounted_return.append(G)
        discounted_return.reverse()
        discounted_return = np.asarray(discounted_return).reshape((-1, 1))
        discounted_return -= np.mean(discounted_return)
        return discounted_return


    def _value_losses(self,advantage):
        return (advantage)**2

    def _policy_losses(self,advantage,selected_action_probs,v,discounted_return):

        a = tf.math.log(selected_action_probs) * advantage
        b = self._entropy(v)
        policy_losses = - ( a + b )

        return policy_losses

    def _entropy(self, v):

        a,_ = v.shape

        ave = v.numpy()    
        sigma2 = np.std(ave)
        entropy = self.beta*0.5*(math.log(2 * math.pi * sigma2) + 1)

        mylist = [[entropy] for i in range(a)]
        rank_1_tensor = tf.constant(mylist)

        return rank_1_tensor

In [6]:
class Main:
    def __init__(self, env, actor, critic, mdl_dir, name, batch_size = 32, episodes_times = 1000, mode = 'test'):
        self.env = env
        self.actor = actor
        self.critic = critic
        self.mdl_dir = mdl_dir
        self.scaler = self._standard_scaler(self.env)
        self.episodes_times = episodes_times
        self.batch_size = batch_size
        self.mode = mode
        self.name = name

        if self.mode == 'test':
            self._load()
            self.actor.epsilon = 0.01
            self.df_rec = pd.DataFrame(index=[], columns=['FixedProfit','TradeTimes','TradeWin'])
        else:
            self.df_rec = pd.DataFrame(index=[], columns=['FixedProfit'])

    def play_game(self):

        for episode in range(self.episodes_times):
            state = self.env.reset()
            state = self.scaler.transform([state])
            state = state.flatten()
            done = False
            start_time = datetime.now()

            total_reward = 0.0
            experiences = []
    
            while not done:
                
                action = self.actor.policynetwork(state)
                next_state, reward, done, info = self.env.step(action)
                next_state = self.scaler.transform([next_state])
                next_state = next_state.flatten()

                total_reward += reward

                if mode == 'train':
                    experiences.append({"state": state, "action": action, "reward": reward, "next_state": next_state, "done": done,})
                    if len(experiences) == self.batch_size:
                        self.critic.valuenetwork(experiences)
                        experiences = []

                state = next_state
               
            play_time = datetime.now() - start_time
            if self.mode == 'test':
                record = pd.Series([info['cur_revenue'],info['trade_time'],info['trade_win']], index=self.df_rec.columns)
                print("Episode: {}/{} RapTime: {} FixedProfit: {:.0f} TradeTimes: {} TradeWin: {}".format(episode + 1, self.episodes_times, play_time, info['cur_revenue'], info['trade_time'], info['trade_win']))
            else:
                record = pd.Series(info['cur_revenue'], index=self.df_rec.columns)
                print("Episode: {}/{} RapTime: {} FixedProfit: {:.0f}".format(episode + 1, self.episodes_times, play_time, info['cur_revenue']))

            self.df_rec = self.df_rec.append(record, ignore_index=True)

        if self.mode == 'train':
            self._save()
        self._save_csv()

    def _standard_scaler(self, env):
        states = []
        for _ in range(env.df_total_steps):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                break
        
        scaler = StandardScaler()
        scaler.fit(states)
        return scaler

    def _load(self):
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'rb') as f:
            self.scaler = pickle.load(f)
        self.actor.load('{}/{}.h5'.format(self.mdl_dir, self.name))

    def _save(self):
        self.actor.save('{}/{}.h5'.format(self.mdl_dir, self.name))
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'wb') as f:
            pickle.dump(self.scaler, f)

    def _save_csv(self):
        self.df_rec.to_csv(csv_path)

In [7]:
initial_money=1000000
episodes_times = 100
batch_size = 32

actor = Actor()
critic = Critic()
env = Environment(df, initial_money, mode)
main = Main(env, actor, critic, mdl_dir, name, batch_size, episodes_times, mode)
main.play_game()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          512         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            387         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            129         dense[0][0]                      
Total params: 1,028
Trainable params: 1,028
Non-trainable params: 0
__________________________